In [ ]:
import yaml

with open('../config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Example usage:
model_params = config['modeling']['regression_model']['params']
features = config['features']['regression']
model_path = config['models']['regression_model']


In [ ]:
# Cell 1: Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import joblib
import warnings
warnings.filterwarnings('ignore')

print("Starting regression model development...")


In [ ]:
# Cell 2: Load processed data
df = pd.read_csv('../data/processed/regression_data.csv')
print(f"Regression data loaded. Shape: {df.shape}")

# Load feature info
import json
with open('../data/processed/feature_info.json', 'r') as f:
    feature_info = json.load(f)

regression_features = feature_info['regression_features']
print(f"Number of features: {len(regression_features)}")


In [ ]:
# Cell 3: Prepare data for modeling
print("=== PREPARING DATA FOR MODELING ===")

# Features and target
X = df[regression_features]
y = df['delay_duration']

# Focus on delayed flights only (delay_duration > 0) for better regression performance
delayed_mask = y > 0
X_delayed = X[delayed_mask]
y_delayed = y[delayed_mask]

print(f"Total records: {len(df):,}")
print(f"Delayed flights: {len(X_delayed):,}")
print(f"Feature matrix shape: {X_delayed.shape}")
print(f"Target statistics:")
print(f"   - Mean delay: {y_delayed.mean():.2f} minutes")
print(f"   - Median delay: {y_delayed.median():.2f} minutes")
print(f"   - Max delay: {y_delayed.max():.2f} minutes")

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_delayed, y_delayed, test_size=0.2, random_state=42
)

print(f"Training set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")


In [ ]:
# Cell 4: Model training - Random Forest Regressor
print("=== TRAINING RANDOM FOREST REGRESSOR ===")

# Initialize Random Forest Regressor
rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=15,
    min_samples_split=10,
    min_samples_leaf=5,
    random_state=42,
    n_jobs=-1
)

# Train the model
rf_model.fit(X_train, y_train)

# Predictions
y_pred_rf = rf_model.predict(X_test)

print("Random Forest Regressor trained successfully!")


In [ ]:
# Cell 5: Model training - Linear Regression
print("=== TRAINING LINEAR REGRESSION ===")

# Initialize Linear Regression
lr_model = LinearRegression()

# Train the model
lr_model.fit(X_train, y_train)

# Predictions
y_pred_lr = lr_model.predict(X_test)

print("Linear Regression model trained successfully!")


In [ ]:
# Cell 6: Model evaluation
print("=== MODEL EVALUATION ===")

# Function to evaluate regression model
def evaluate_regression_model(y_true, y_pred, model_name):
    print(f"\n{model_name} Performance:")
    print("-" * 40)
    
    # Calculate metrics
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    
    # Additional metrics
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100  # Mean Absolute Percentage Error
    
    print(f"MAE (Mean Absolute Error):     {mae:.2f} minutes")
    print(f"RMSE (Root Mean Squared Error): {rmse:.2f} minutes")
    print(f"R² Score:                      {r2:.4f}")
    print(f"MAPE (Mean Absolute % Error):  {mape:.2f}%")
    
    return {
        'mae': mae, 'rmse': rmse, 'r2': r2, 'mape': mape
    }

# Evaluate both models
rf_metrics = evaluate_regression_model(y_test, y_pred_rf, "Random Forest Regressor")
lr_metrics = evaluate_regression_model(y_test, y_pred_lr, "Linear Regression")


In [ ]:
# Cell 7: Visualization of results
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Random Forest: Actual vs Predicted
axes[0,0].scatter(y_test, y_pred_rf, alpha=0.5, color='blue')
axes[0,0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[0,0].set_xlabel('Actual Delay (minutes)')
axes[0,0].set_ylabel('Predicted Delay (minutes)')
axes[0,0].set_title(f'Random Forest: Actual vs Predicted\n(R² = {rf_metrics["r2"]:.3f})')
axes[0,0].grid(True, alpha=0.3)

# Linear Regression: Actual vs Predicted
axes[0,1].scatter(y_test, y_pred_lr, alpha=0.5, color='green')
axes[0,1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[0,1].set_xlabel('Actual Delay (minutes)')
axes[0,1].set_ylabel('Predicted Delay (minutes)')
axes[0,1].set_title(f'Linear Regression: Actual vs Predicted\n(R² = {lr_metrics["r2"]:.3f})')
axes[0,1].grid(True, alpha=0.3)

# Residuals plot - Random Forest
residuals_rf = y_test - y_pred_rf
axes[1,0].scatter(y_pred_rf, residuals_rf, alpha=0.5, color='blue')
axes[1,0].axhline(y=0, color='r', linestyle='--')
axes[1,0].set_xlabel('Predicted Delay (minutes)')
axes[1,0].set_ylabel('Residuals')
axes[1,0].set_title('Random Forest: Residuals Plot')
axes[1,0].grid(True, alpha=0.3)

# Feature Importance (Random Forest)
feature_importance = pd.DataFrame({
    'feature': regression_features,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False).head(15)

axes[1,1].barh(range(len(feature_importance)), feature_importance['importance'])
axes[1,1].set_yticks(range(len(feature_importance)))
axes[1,1].set_yticklabels(feature_importance['feature'])
axes[1,1].set_xlabel('Feature Importance')
axes[1,1].set_title('Top 15 Feature Importances (Random Forest)')

plt.tight_layout()
plt.savefig('../visualizations/model_results/regression_model_evaluation.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Cell 8: Error distribution analysis
plt.figure(figsize=(15, 5))

# Error distribution - Random Forest
plt.subplot(1, 3, 1)
plt.hist(residuals_rf, bins=50, alpha=0.7, color='blue', edgecolor='black')
plt.title('Random Forest: Error Distribution')
plt.xlabel('Prediction Error (minutes)')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

# Error distribution - Linear Regression
residuals_lr = y_test - y_pred_lr
plt.subplot(1, 3, 2)
plt.hist(residuals_lr, bins=50, alpha=0.7, color='green', edgecolor='black')
plt.title('Linear Regression: Error Distribution')
plt.xlabel('Prediction Error (minutes)')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

# Model comparison
plt.subplot(1, 3, 3)
metrics_comparison = pd.DataFrame({
    'Random Forest': [rf_metrics['mae'], rf_metrics['rmse'], rf_metrics['r2']*100],
    'Linear Regression': [lr_metrics['mae'], lr_metrics['rmse'], lr_metrics['r2']*100]
}, index=['MAE', 'RMSE', 'R² (×100)'])

metrics_comparison.plot(kind='bar', ax=plt.gca())
plt.title('Model Performance Comparison')
plt.ylabel('Metric Value')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../visualizations/model_results/regression_error_analysis.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Cell 9: SHAP Analysis for Regression
print("=== SHAP ANALYSIS FOR REGRESSION ===")

# Create SHAP explainer for Random Forest (assuming it's the better model)
explainer = shap.TreeExplainer(rf_model)
shap_values = explainer.shap_values(X_test.iloc[:1000])  # Use subset for speed

# SHAP summary plot
plt.figure(figsize=(12, 8))
shap.summary_plot(shap_values, X_test.iloc[:1000], show=False)
plt.title('SHAP Summary Plot - Regression Model')
plt.tight_layout()
plt.savefig('../visualizations/model_results/shap_summary_regression.png', dpi=300, bbox_inches='tight')
plt.show()

# SHAP feature importance
plt.figure(figsize=(10, 8))
shap.summary_plot(shap_values, X_test.iloc[:1000], plot_type="bar", show=False)
plt.title('SHAP Feature Importance - Regression Model')
plt.tight_layout()
plt.savefig('../visualizations/model_results/shap_importance_regression.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Cell 10: OAI (Operational Adjustability Index) for Regression
print("=== OPERATIONAL ADJUSTABILITY INDEX (OAI) FOR REGRESSION ===")

def calculate_oai_regression(X, y_true, y_pred):
    """
    Calculate OAI for regression model
    OAI prioritizes controllable delays (carrier, late_aircraft)
    """
    # Define controllable and uncontrollable features
    controllable_features = ['carrier_delay', 'late_aircraft_delay', 'controllable_delay_minutes']
    controllable_weight = 3.0
    
    uncontrollable_features = ['weather_delay', 'nas_delay', 'security_delay']
    uncontrollable_weight = 1.0
    
    # Calculate weighted errors
    oai_errors = []
    
    for idx in range(len(X)):
        row = X.iloc[idx]
        actual = y_true.iloc[idx]
        predicted = y_pred[idx]
        base_error = abs(actual - predicted)
        
        # Calculate controllable factor
        controllable_factor = 0
        for feature in controllable_features:
            if feature in X.columns and row[feature] > 0:
                controllable_factor += controllable_weight
        
        # Calculate uncontrollable factor  
        uncontrollable_factor = 0
        for feature in uncontrollable_features:
            if feature in X.columns and row[feature] > 0:
                uncontrollable_factor += uncontrollable_weight
        
        # Weight the error based on controllability
        total_weight = controllable_factor + uncontrollable_factor
        if total_weight > 0:
            controllable_ratio = controllable_factor / total_weight
            # Higher weight for controllable delays (we want to minimize these errors more)
            weighted_error = base_error * (1 + controllable_ratio)
        else:
            weighted_error = base_error
            
        oai_errors.append(weighted_error)
    
    return np.array(oai_errors)

# Calculate OAI metrics
oai_errors_rf = calculate_oai_regression(X_test, y_test, y_pred_rf)
standard_mae_rf = mean_absolute_error(y_test, y_pred_rf)
oai_mae_rf = np.mean(oai_errors_rf)

print(f"Random Forest:")
print(f"Standard MAE: {standard_mae_rf:.2f} minutes")
print(f"OAI-weighted MAE: {oai_mae_rf:.2f} minutes")
print(f"OAI emphasizes controllable delay errors")

# Calculate OAI for Linear Regression too
oai_errors_lr = calculate_oai_regression(X_test, y_test, y_pred_lr)
standard_mae_lr = mean_absolute_error(y_test, y_pred_lr)
oai_mae_lr = np.mean(oai_errors_lr)

print(f"\nLinear Regression:")
print(f"Standard MAE: {standard_mae_lr:.2f} minutes")
print(f"OAI-weighted MAE: {oai_mae_lr:.2f} minutes")


In [ ]:
# Cell 11: Model Selection and Saving
print("=== MODEL SELECTION AND SAVING ===")

# Select best model based on R² score
if rf_metrics['r2'] > lr_metrics['r2']:
    best_model = rf_model
    best_model_name = "Random Forest Regressor"
    best_metrics = rf_metrics
    best_predictions = y_pred_rf
    best_oai_mae = oai_mae_rf
else:
    best_model = lr_model
    best_model_name = "Linear Regression"
    best_metrics = lr_metrics
    best_predictions = y_pred_lr
    best_oai_mae = oai_mae_lr

print(f"Best model: {best_model_name}")
print(f"Best R² Score: {best_metrics['r2']:.4f}")
print(f"Best MAE: {best_metrics['mae']:.2f} minutes")

# Save the best model
import os
os.makedirs('../data/models/trained_models', exist_ok=True)

joblib.dump(best_model, '../data/models/trained_models/regression_model.pkl')
joblib.dump(explainer, '../data/models/trained_models/regression_explainer.pkl')

# Save model performance metrics
model_results = {
    'model_type': 'regression',
    'best_model': best_model_name,
    'metrics': {
        'random_forest': rf_metrics,
        'linear_regression': lr_metrics
    },
    'oai_metrics': {
        'random_forest_oai_mae': float(oai_mae_rf),
        'linear_regression_oai_mae': float(oai_mae_lr),
        'best_model_oai_mae': float(best_oai_mae)
    },
    'feature_count': len(regression_features),
    'test_size': len(X_test),
    'delayed_flights_only': True
}

import json
with open('../data/models/trained_models/regression_results.json', 'w') as f:
    json.dump(model_results, f, indent=2)

print("Regression model saved successfully!")
print(f"Model file: regression_model.pkl")
print(f"Results file: regression_results.json")


In [ ]:
# Cell 12: Final Summary
print("=== REGRESSION MODEL SUMMARY ===")
print(f"✅ Model Type: Flight Delay Duration Prediction (Minutes)")
print(f"✅ Best Model: {best_model_name}")
print(f"✅ Dataset Size: {len(X_delayed):,} delayed flights")
print(f"✅ Features Used: {len(regression_features)}")
print(f"✅ Test Set Performance:")
print(f"   - MAE:  {best_metrics['mae']:.2f} minutes")
print(f"   - RMSE: {best_metrics['rmse']:.2f} minutes")
print(f"   - R²:   {best_metrics['r2']:.4f}")
print(f"   - MAPE: {best_metrics['mape']:.2f}%")
print(f"✅ OAI Analysis: Completed (OAI MAE: {best_oai_mae:.2f} minutes)")
print(f"✅ SHAP Analysis: Completed")
print(f"✅ Model Saved: ../data/models/trained_models/regression_model.pkl")
